In [ ]:
import pickle
import os

from scipy.integrate import quad

import StaticWindow as sw

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

# Load Data

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.
    
background_file = '0_SNR_data.pkl' # Baseline eventless data filename.

files = [] # List of simulation filenames

SNRs = [float(x.split('_')[0]) for x in files]

# Load Data and Ground Truths
raw_traces, ground_truths = sw.load_SNR_data(directory, files)

thresholds = [0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999, 1]

# Load Background data
with open(os.path.join(directory, background_file), 'rb') as f:
    background_data = pickle.load(f)[0]

# Analyse Background

In [ ]:
background_data_scaled = background_data * 1000

In [ ]:
# Calculate the Gaussian Distribution of Background ZCR values.
background_results = sw.analyse_background(background_data_scaled, 100, 10)
means, bin_mids, bin_counts, gauss_params, norm_factor = background_results

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
ax.bar(bin_mids, bin_counts / norm_factor, np.diff(bin_mids)[0])

x = np.linspace(-0.5, 0.5, 100)
y = sw.gauss(x, *gauss_params)

ax.plot(x, y / norm_factor, color='red')
ax.set_xlabel("Current / pA", weight='bold')
ax.set_ylabel("Nomralised Counts", weight='bold')

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)

text_str = "Gaussian\nMean: {:.2f}\nStd: {:.2f}".format(gauss_params[1], gauss_params[2])
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
ax.text(0.05, 0.95, text_str, transform=ax.transAxes,
        verticalalignment='top', bbox=props)

# fig.tight_layout()
fig.savefig('MA Background.png')

In [ ]:
# Calculate the Threshold Values from the Distribution.
gauss_params_norm = gauss_params.copy()
gauss_params_norm[0] = gauss_params_norm[0] / norm_factor

mean_thresholds = []
for thresh in thresholds:
    print(thresh)
    val = sw.limit_finder(gauss_params_norm, means.min(), means.max(), thresh)
    mean_thresholds.append(val)
thresh_dict = dict(zip(thresholds, mean_thresholds))
thresh_dict

# Event Search

## Find Anomalies

In [ ]:
window_size = 10
step_size = 1

In [ ]:
columns = ['SNR', 'Thresh', 'event_locations']
df = pd.DataFrame(columns=columns)

for snr in SNRs:
    for thresh in thresholds:
        print("SNR: " + str(snr))
        print("Threshold: " + str(thresh))
        
        # Get threshold value
        thresh_val = thresh_dict[thresh] / 1000
        
        traces = raw_traces[snr]
        
        # Locate Events
        evt_locs = sw.event_search(traces, thresh_val, window_size, step_size)
        
        print("Event Search Finished")
        
        df2 = pd.DataFrame([[snr, thresh, evt_locs]], columns=columns)
        df = pd.concat((df, df2))

In [ ]:
df = df.set_index(['SNR', 'Thresh'])
df

# Save Found Events

In [ ]:
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(df, f)